### 一、验证码图片素材生成

#### 教程中网站过期了，本想尝试苏州大学教务系统的登录页，但是那个验证码是由数字、大小写字母组成的，数据处理时工作量太大，而本实验的重点不在数据获取这块，我就换用了 Pillow 来生成验证码图片素材。


In [37]:
# === 步骤一 (v4): 生成“中等难度”(仅含噪点)的图片 ===

from PIL import Image, ImageDraw, ImageFont
import random
import os
import shutil

# --- 0. 定义参数 (★ 回到“固定”坐标 ★) ---
CHAR_WIDTH = 8
CHAR_HEIGHT = 18
CHAR_SPACING = 5
START_X = 2
START_Y = 0
IMG_WIDTH = 52
IMG_HEIGHT = 18

TARGET_FOLDER = 'medium_images' # ★ 新文件夹
NUM_IMAGES = 1000

# --- 1. 准备工作 ---
if os.path.exists(TARGET_FOLDER):
    shutil.rmtree(TARGET_FOLDER)
os.makedirs(TARGET_FOLDER)

try:
    font = ImageFont.truetype("arial.ttf", 16)
except IOError:
    print("未找到 Arial 字体, 使用Pillow默认字体")
    font = ImageFont.load_default()

print(f"开始在 {TARGET_FOLDER} 中生成 {NUM_IMAGES} 张“中等难度”的图片...")

# --- 2. 循环生成图片 ---
for i in range(NUM_IMAGES):
    
    label = ''.join([random.choice('0123456789') for _ in range(4)])
    img = Image.new('L', (IMG_WIDTH, IMG_HEIGHT), color=255)
    draw = ImageDraw.Draw(img)

    # 3. ★ 关键：我们用回“固定”坐标，不再旋转和偏移 ★
    for j in range(4):
        char = label[j]
        x = START_X + j * (CHAR_WIDTH + CHAR_SPACING)
        y = START_Y
        draw.text((x, y), char, fill=0, font=font) # 黑色

    # 4. ★ 关键：我们只增加“噪点”的数量 ★
    for k in range(120): # 之前是 10 或 25, 我们增加到 120
        nx = random.randint(0, IMG_WIDTH - 1)
        ny = random.randint(0, IMG_HEIGHT - 1)
        # 50% 概率画黑点, 50% 概率画白点 (挖洞)
        if random.random() > 0.5:
            draw.point((nx, ny), fill=0) # 黑点
        else:
            draw.point((nx, ny), fill=255) # 白点 (挖洞)

    # 5. 保存图片
    filename = os.path.join(TARGET_FOLDER, f'{label}_{i}.png')
    img.save(filename)

print(f"--- {NUM_IMAGES} 张图片已生成！ ---")

开始在 medium_images 中生成 1000 张“中等难度”的图片...
--- 1000 张图片已生成！ ---


### 二、图像处理与切割


In [38]:
# === 步骤二 (v4): 使用“固定坐标”切割 ===

# ★ 修正：更新文件夹名 ★
GENERATED_IMG_FOLDER = 'medium_images'
LABELED_DATA_FOLDER = 'training_dataset'

def get_bin_table(threshold=140):
    table = []
    for i in range(256):
        if i < threshold:
            table.append(0)
        else:
            table.append(1)
    return table

def get_crop_imgs(img_binary):
    """
    ★ 关键：我们用回教程的“固定坐标”切割
    """
    child_img_list = []
    for i in range(4):
        x = 2 + i * (8 + 5)
        y = 0
        child_img = img_binary.crop((x, y, x + 8, y + 18))
        child_img_list.append(child_img)
    return child_img_list

def process_and_label_images():
    print(f"\n--- 步骤二：开始处理图片并自动打标 ---")
    
    if not os.path.exists(GENERATED_IMG_FOLDER):
        print(f"错误：找不到 {GENERATED_IMG_FOLDER}。")
        return

    if os.path.exists(LABELED_DATA_FOLDER):
        shutil.rmtree(LABELED_DATA_FOLDER)
    os.makedirs(LABELED_DATA_FOLDER)

    binary_table = get_bin_table()
    total_images = 0

    for img_name in os.listdir(GENERATED_IMG_FOLDER):
        if not img_name.endswith('.png'):
            continue
        
        label = img_name.split('_')[0]
        img_path = os.path.join(GENERATED_IMG_FOLDER, img_name)
        img = Image.open(img_path)
        img_gray = img.convert('L')
        img_binary = img_gray.point(binary_table, '1')

        # ★ 关键：调用“固定坐标”切割 ★
        child_images = get_crop_imgs(img_binary)

        if len(child_images) != len(label):
            continue
            
        for i in range(len(label)):
            char_label = label[i]
            child_img = child_images[i]
            char_folder = os.path.join(LABELED_DATA_FOLDER, char_label)
            if not os.path.exists(char_folder):
                os.makedirs(char_folder)
            save_name = f"{img_name.split('.')[0]}_{i}.png"
            child_img.save(os.path.join(char_folder, save_name))
            total_images += 1

    print(f"--- 步骤二：完成！共 {total_images} 张切割好的图片已存入 {LABELED_DATA_FOLDER} ---")

# --- 运行步骤二 ---
process_and_label_images()


--- 步骤二：开始处理图片并自动打标 ---
--- 步骤二：完成！共 4000 张切割好的图片已存入 training_dataset ---


### 三、提取特征，生成 libsvm 格式文件


In [39]:
# === 步骤三 (v4): 使用“原始特征”(行列求和) ===

LABELED_DATA_FOLDER = 'training_dataset'
TRAIN_FILE = 'train_feature.txt'
TEST_FILE = 'test_feature.txt'

def get_feature(img):
    """
    ★ 关键：我们用回教程的“行列求和”特征
    """
    try:
        # 尺寸 (8, 18)
        img = img.resize((8, 18))
    except Exception as e:
        return None
        
    width, height = img.size
    pixel_cnt_list = []

    # 横向特征
    for y in range(height): # 18
        pix_cnt_x = 0
        for x in range(width): # 8
            if img.getpixel((x, y)) == 0:
                pix_cnt_x += 1
        pixel_cnt_list.append(pix_cnt_x)

    # 纵向特征
    for x in range(width): # 8
        pix_cnt_y = 0
        for y in range(height): # 18
            if img.getpixel((x, y)) == 0: 
                pix_cnt_y += 1
        pixel_cnt_list.append(pix_cnt_y)
    
    # (18 + 8 = 26) 维
    return pixel_cnt_list

def create_feature_files():
    print(f"\n--- 步骤三：开始提取特征并生成 LibSVM 格式文件 ---")
    
    if os.path.exists(TRAIN_FILE):
        os.remove(TRAIN_FILE)
    if os.path.exists(TEST_FILE):
        os.remove(TEST_FILE)

    f_train = open(TRAIN_FILE, 'a')
    f_test = open(TEST_FILE, 'a')
    
    total_train = 0
    total_test = 0

    for n in range(10): 
        char_label = str(n)
        char_folder = os.path.join(LABELED_DATA_FOLDER, char_label)
        if not os.path.exists(char_folder):
            continue
        img_list = os.listdir(char_folder)
        split_index = int(len(img_list) * 0.8)
        
        for i, im_file in enumerate(img_list):
            im_path = os.path.join(char_folder, im_file)
            try:
                im = Image.open(im_path)
            except:
                continue
            im_feature = get_feature(im)
            if im_feature is None:
                continue
            line_parts = [char_label]
            for z_index, z_value in enumerate(im_feature):
                line_parts.append(f"{z_index + 1}:{z_value}")
            line = ' '.join(line_parts) + '\n'
            
            if i < split_index:
                f_train.write(line)
                total_train += 1
            else:
                f_test.write(line)
                total_test += 1

    f_train.close()
    f_test.close()

    print(f"--- 步骤三：完成！ ---")
    print(f"训练集文件已保存: {TRAIN_FILE} (共 {total_train} 条数据)")
    print(f"测试集文件已保存: {TEST_FILE} (共 {total_test} 条数据)")

# --- 运行步骤三 ---
create_feature_files()


--- 步骤三：开始提取特征并生成 LibSVM 格式文件 ---
--- 步骤三：完成！ ---
训练集文件已保存: train_feature.txt (共 3196 条数据)
测试集文件已保存: test_feature.txt (共 804 条数据)


### 四、训练模型（利用不同核函数：线性核、多项式核、RBF 核、Sigmoid 核）


In [41]:
# === 步骤四：训练模型并对比核函数 (实验核心) ===
import os, sys, importlib.util

TRAIN_FILE = 'train_feature.txt'
TEST_FILE = 'test_feature.txt'

# 兼容两种安装（优先 libsvm-official）
try:
    from libsvm.svmutil import svm_read_problem, svm_train, svm_predict
    print("★★★ 成功从 libsvm.svmutil 导入！（libsvm-official）★★★")
except ModuleNotFoundError:
    try:
        from svmutil import svm_read_problem, svm_train, svm_predict
        print("★★★ 成功从顶层 svmutil 导入！（libsvm 包）★★★")
    except ModuleNotFoundError as e:
        print("\n--- 仍然导入失败 ---")
        print("你当前的 Python 可执行文件：", sys.executable)
        print("find_spec('libsvm.svmutil') =", importlib.util.find_spec('libsvm.svmutil'))
        print("find_spec('svmutil')        =", importlib.util.find_spec('svmutil'))
        print("若你安装的是 libsvm-official，请使用：from libsvm.svmutil import ...")
        raise

def train_and_compare_kernels():
    print(f"\n--- 步骤四：开始训练模型并对比核函数 ---")
    if not os.path.exists(TRAIN_FILE) or not os.path.exists(TEST_FILE):
        print(f"错误：找不到 {TRAIN_FILE} 或 {TEST_FILE}。请先运行步骤三。")
        return

    try:
        y_train, x_train = svm_read_problem(TRAIN_FILE)
        y_test, x_test = svm_read_problem(TEST_FILE)
    except Exception as e:
        print(f"错误：读取特征文件失败。{e}")
        return

    if not y_train or not y_test:
        print("错误：训练集或测试集为空。")
        return

    kernels = {
        '线性核 (Linear, -t 0)': '-t 0 -q',
        '多项式核 (Poly, -t 1)': '-t 1 -q',
        'RBF 核 (Gaussian, -t 2)': '-t 2 -q',
        'Sigmoid 核 (-t 3)': '-t 3 -q'
    }
    results = {}

    print("\n--- ★★★ 开始执行实验 ★★★ ---")
    for kernel_name, kernel_param in kernels.items():
        print(f"\n正在训练 {kernel_name}...")
        model = svm_train(y_train, x_train, kernel_param)
        print(f"正在测试 {kernel_name}...")
        p_labels, p_acc, p_vals = svm_predict(y_test, x_test, model)
        accuracy = p_acc[0]
        results[kernel_name] = accuracy
        print(f"--- {kernel_name} 的准确率 (Accuracy): {accuracy:.2f}% ---")

    print("\n\n--- ★★★ 最终实验对比结果 ★★★ ---")
    for kernel_name, accuracy in results.items():
        print(f"{kernel_name.ljust(25)}: {accuracy:.2f}%")
    print("---------------------------------")
    print("\n--- 实验任务全部完成！ ---")

# 运行
train_and_compare_kernels()


★★★ 成功从 libsvm.svmutil 导入！（libsvm-official）★★★

--- 步骤四：开始训练模型并对比核函数 ---

--- ★★★ 开始执行实验 ★★★ ---

正在训练 线性核 (Linear, -t 0)...
正在测试 线性核 (Linear, -t 0)...
Accuracy = 99.2537% (798/804) (classification)
--- 线性核 (Linear, -t 0) 的准确率 (Accuracy): 99.25% ---

正在训练 多项式核 (Poly, -t 1)...
正在测试 多项式核 (Poly, -t 1)...
Accuracy = 99.2537% (798/804) (classification)
--- 多项式核 (Poly, -t 1) 的准确率 (Accuracy): 99.25% ---

正在训练 RBF 核 (Gaussian, -t 2)...
正在测试 RBF 核 (Gaussian, -t 2)...
Accuracy = 99.8756% (803/804) (classification)
--- RBF 核 (Gaussian, -t 2) 的准确率 (Accuracy): 99.88% ---

正在训练 Sigmoid 核 (-t 3)...
正在测试 Sigmoid 核 (-t 3)...
Accuracy = 10.5721% (85/804) (classification)
--- Sigmoid 核 (-t 3) 的准确率 (Accuracy): 10.57% ---


--- ★★★ 最终实验对比结果 ★★★ ---
线性核 (Linear, -t 0)       : 99.25%
多项式核 (Poly, -t 1)        : 99.25%
RBF 核 (Gaussian, -t 2)   : 99.88%
Sigmoid 核 (-t 3)         : 10.57%
---------------------------------

--- 实验任务全部完成！ ---
